In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [2]:
df = pd.read_csv("C:/Users/limst/Documents/Introduction to Machine Learning/Project/Data/train_healthy.csv")
df

,recipe_id,groups,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,5315,2,beverages,icy holiday punch,6,1440,54.5,0.0,50.0,0.0,...,3.912023,0.000000,0.000000,0.000000,1.386294,8629,2001-04-24,I brought this to an office holiday party and ...,4,.
1,8661,1,salads,oliveau salad dressing,8,20,168.7,28.0,3.0,3.0,...,1.098612,1.098612,0.000000,2.484907,0.000000,13483,2001-08-05,Good recipe and easy to make. I found it a li...,4,traditional french dressing -- this recipe was...
2,10859,2,vegetables,broccoli cauliflower and rice casserole,5,85,228.8,24.0,12.0,46.0,...,2.484907,3.828641,2.890372,3.465736,1.386294,8629,2001-08-15,This is a nice casserole. It is very easy to ...,2,easy and quick to prepare. grandmother's recip...
3,902,2,vegetables,microwaved garlic mushroom rolls,7,0,1188.6,88.0,53.0,69.0,...,3.970292,4.234107,4.317488,5.030438,3.828641,13483,2001-08-16,Very easy to make for a lunch. I served it co...,3,"i use brown flat mushrooms and slice them, i f..."
4,8430,2,desserts,lemon blueberry bundt cake,10,70,234.0,13.0,89.0,10.0,...,4.488636,2.302585,1.791759,3.258097,2.484907,13483,2001-09-04,I made this and froze it. I served it a week ...,4,delicious bundt cake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65179,268004,1,soup,mashed potato and turkey soup,9,30,243.7,12.0,13.0,18.0,...,2.564949,2.890372,3.663562,2.890372,2.079442,125388,2010-04-09,"This was good, and gives a lot of flavor for h...",4,a simple recipe for leftovers. souper easy an...
65180,145707,1,soup,becky s taco soup,9,75,476.2,12.0,2.0,23.0,...,0.693147,3.135494,4.158883,2.639057,3.178054,28177,2010-04-09,"Becky's a good friend to have around, farmwife...",5,this is the soup that my friend becky brings w...
65181,89292,1,vegetables,committee salad,14,15,301.5,43.0,18.0,7.0,...,2.890372,1.945910,2.197225,3.178054,1.098612,333017,2010-04-09,I found this recipe in the Best of Bridge seri...,4,from the best of bridge series - winners book
65182,291504,1,vegetables,mustard crusted roast new potatoes with shallo...,7,55,201.0,15.0,4.0,0.0,...,1.386294,0.000000,1.791759,1.945910,2.079442,192581,2010-04-09,"Okay, instead of the coarse grain mustard I us...",5,a flavour packed roast potato recipe which res...


# One-hot encoding of the category (Normalised data)

In [3]:
def train_val_split(df):

    import math

    date_80th = df.iloc[math.floor(.8 * len(df))]['date']
    print('Date of the 80th-percentile review: ', date_80th)  

    i_date = df.index[df['date'] == date_80th].tolist()
    print(f'Index of the first review on {date_80th}: ', i_date[0]) 

    train_set = df.iloc[:i_date[0]]
    print(train_set.shape)

    val_set = df.iloc[i_date[0]:]
    print(val_set.shape)

    return (train_set, val_set)

train_set, val_set = train_val_split(df)

Date of the 80th-percentile review:  2009-02-15
Index of the first review on 2009-02-15:  52138
(52138, 25)
(13046, 25)


In [4]:
X_train = train_set[['category', 'log calories', 'log fat',
       'log sugar', 'log sodium', 'log protein', 'log sat fat', 'log carbs']]
X_train = pd.concat([X_train, pd.get_dummies(X_train.category, prefix='Cat_')], axis=1).drop('category', axis = 1)
X_train

,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs,Cat__beverages,Cat__breads,Cat__burgers,Cat__desserts,Cat__meat,Cat__pasta,Cat__salads,Cat__seafood,Cat__soup,Cat__vegetables
0,3.998201,0.000000,3.912023,0.000000,0.000000,0.000000,1.386294,1,0,0,0,0,0,0,0,0,0
1,5.128122,3.332205,1.098612,1.098612,0.000000,2.484907,0.000000,0,0,0,0,0,0,1,0,0,0
2,5.432848,3.178054,2.484907,3.828641,2.890372,3.465736,1.386294,0,0,0,0,0,0,0,0,0,1
3,7.080531,4.477337,3.970292,4.234107,4.317488,5.030438,3.828641,0,0,0,0,0,0,0,0,0,1
4,5.455321,2.564949,4.488636,2.302585,1.791759,3.258097,2.484907,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52133,5.735281,3.218876,3.091042,3.258097,3.663562,3.806662,1.945910,0,0,0,0,0,0,0,0,0,1
52134,4.553877,2.484907,0.000000,2.197225,2.197225,3.218876,0.000000,0,0,0,0,0,0,0,0,0,1
52135,6.896087,4.394449,3.637586,4.795791,4.644391,4.859812,3.218876,0,0,0,0,1,0,0,0,0,0
52136,5.813533,2.302585,2.302585,3.044522,4.262680,1.609438,2.302585,0,0,0,0,0,0,0,0,0,1


In [5]:
y_train = train_set['groups']
y_train

0        2
1        1
2        2
3        2
4        2
        ..
52133    1
52134    1
52135    3
52136    1
52137    3
Name: groups, Length: 52138, dtype: int64

In [6]:
X_test = val_set[['category', 'log calories', 'log fat',
       'log sugar', 'log sodium', 'log protein', 'log sat fat', 'log carbs']]
X_test = pd.concat([X_test, pd.get_dummies(X_test.category, prefix='Cat_')], axis=1).drop('category', axis = 1)
X_test

,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs,Cat__beverages,Cat__breads,Cat__burgers,Cat__desserts,Cat__meat,Cat__pasta,Cat__salads,Cat__seafood,Cat__soup,Cat__vegetables
52138,4.768988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,0,0,0,0,0,0,0,0
52139,6.354196,3.912023,3.295837,3.091042,3.828641,4.543295,2.708050,0,1,0,0,0,0,0,0,0,0
52140,5.833055,3.332205,4.521789,2.564949,2.197225,3.044522,2.564949,0,0,0,1,0,0,0,0,0,0
52141,7.217590,4.882802,5.105945,3.496508,5.056246,4.997212,3.044522,0,0,0,0,1,0,0,0,0,0
52142,5.799699,2.708050,2.197225,2.708050,4.430817,2.197225,1.609438,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65179,5.495938,2.484907,2.564949,2.890372,3.663562,2.890372,2.079442,0,0,0,0,0,0,0,0,1,0
65180,6.165838,2.484907,0.693147,3.135494,4.158883,2.639057,3.178054,0,0,0,0,0,0,0,0,1,0
65181,5.708770,3.761200,2.890372,1.945910,2.197225,3.178054,1.098612,0,0,0,0,0,0,0,0,0,1
65182,5.303305,2.708050,1.386294,0.000000,1.791759,1.945910,2.079442,0,0,0,0,0,0,0,0,0,1


In [7]:
y_test = val_set['groups']
y_test

52138    1
52139    2
52140    2
52141    2
52142    1
        ..
65179    1
65180    1
65181    1
65182    1
65183    1
Name: groups, Length: 13046, dtype: int64

## Multinomial Logistic Regression

In [8]:
# Import 'LogisticRegression' and create a LogisticRegression object
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
# Import RFE and select 15 variables          
rfe = logreg.fit(X_train, y_train)

C:\Users\limst\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
y_pred = rfe.predict(X_test)
print('Accuracy', accuracy_score(y_test,y_pred)*100,'%')

Accuracy 77.94726352905104 %


# One vs One classification

## Linear 

In [10]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
linear_pred = linear.predict(X_test)

## Polynomial Kernal

In [11]:
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(X_train, y_train)
poly_pred = poly.predict(X_test)

## RBF Kernel

In [12]:
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)

## Sigmoid kernel

In [13]:
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)
sig_pred = sig.predict(X_test)

## Compare results

In [14]:
accuracy_lin_ovo_lg = linear.score(X_test, y_test)
accuracy_poly_ovo_lg = poly.score(X_test, y_test)
accuracy_rbf_ovo_lg = rbf.score(X_test, y_test)
accuracy_sig_ovo_lg = sig.score(X_test, y_test)
print('Accuracy Linear Kernel:', accuracy_lin_ovo_lg)
print('Accuracy Polynomial Kernel:', accuracy_poly_ovo_lg)
print('Accuracy Radial Basis Kernel:', accuracy_rbf_ovo_lg)
print('Accuracy Sigmoid Kernel:', accuracy_sig_ovo_lg)

Accuracy Linear Kernel: 0.7949563084470336
Accuracy Polynomial Kernel: 0.9052583167254331
Accuracy Radial Basis Kernel: 0.9486432623026215
Accuracy Sigmoid Kernel: 0.40556492411467115


In [15]:
cm_lin = confusion_matrix(y_test, linear_pred)
cm_poly = confusion_matrix(y_test, poly_pred)
cm_rbf = confusion_matrix(y_test, rbf_pred)
cm_sig = confusion_matrix(y_test, sig_pred)
print('Linear:\n', cm_lin)
print('Polynomial:\n', cm_poly)
print('RBF Kernel:\n', cm_rbf)
print('Sigmoid:\n', cm_sig)

Linear:
 [[5974  887   57]
 [ 973 2940  306]
 [  50  402 1457]]
Polynomial:
 [[6483  424   11]
 [ 512 3603  104]
 [  10  175 1724]]
RBF Kernel:
 [[6732  184    2]
 [ 291 3860   68]
 [  44   81 1784]]
Sigmoid:
 [[5044    0 1874]
 [3527    0  692]
 [1662    0  247]]


# One vs Rest classification

## Linear 

In [16]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovr').fit(X_train, y_train)
linear_pred = linear.predict(X_test)

## Polynomial Kernal

In [17]:
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovr').fit(X_train, y_train)
poly_pred = poly.predict(X_test)

## RBF Kernel

In [18]:
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovr').fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)

## Sigmoid kernel

In [19]:
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovr').fit(X_train, y_train)
sig_pred = sig.predict(X_test)

## Compare results

In [20]:
accuracy_lin_ovr_lg = linear.score(X_test, y_test)
accuracy_poly_ovr_lg = poly.score(X_test, y_test)
accuracy_rbf_ovr_lg = rbf.score(X_test, y_test)
accuracy_sig_ovr_lg = sig.score(X_test, y_test)
print('Accuracy Linear Kernel:', accuracy_lin_ovr_lg)
print('Accuracy Polynomial Kernel:', accuracy_poly_ovr_lg)
print('Accuracy Radial Basis Kernel:', accuracy_rbf_ovr_lg)
print('Accuracy Sigmoid Kernel:', accuracy_sig_ovr_lg)

Accuracy Linear Kernel: 0.7949563084470336
Accuracy Polynomial Kernel: 0.9052583167254331
Accuracy Radial Basis Kernel: 0.9486432623026215
Accuracy Sigmoid Kernel: 0.40556492411467115


In [21]:
cm_lin = confusion_matrix(y_test, linear_pred)
cm_poly = confusion_matrix(y_test, poly_pred)
cm_rbf = confusion_matrix(y_test, rbf_pred)
cm_sig = confusion_matrix(y_test, sig_pred)
print('Linear:\n', cm_lin)
print('Polynomial:\n', cm_poly)
print('RBF Kernel:\n', cm_rbf)
print('Sigmoid:\n', cm_sig)

Linear:
 [[5974  887   57]
 [ 973 2940  306]
 [  50  402 1457]]
Polynomial:
 [[6483  424   11]
 [ 512 3603  104]
 [  10  175 1724]]
RBF Kernel:
 [[6732  184    2]
 [ 291 3860   68]
 [  44   81 1784]]
Sigmoid:
 [[5044    0 1874]
 [3527    0  692]
 [1662    0  247]]


## Selecting the best model: Radial Basis Kernel

In [22]:
X = df[['category', 'log calories', 'log fat',
       'log sugar', 'log sodium', 'log protein', 'log sat fat', 'log carbs']]
X = pd.concat([X, pd.get_dummies(X.category, prefix='Cat_')], axis=1).drop('category', axis = 1)
y = df['groups']

In [23]:
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovr').fit(X, y)

## Using our best fit model to predict healthy recipes for test dataset

In [24]:
test_data = pd.read_csv("C:/Users/limst/Documents/Introduction to Machine Learning/Project/Data/test_dataset_all.csv")

In [25]:
test_data.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,336446,desserts,towboat brownies,8,40,363.4,25.0,135.0,14.0,10.0,...,4.905275,2.639057,2.302585,3.713572,2.772589,526666,2010-04-10,"Cakey goodness, and oh so simple. I creamed th...",5,from the new orleans times picayune
1,47441,salads,tsr version of islands china coast salad dress...,6,7,101.2,12.0,16.0,9.0,0.0,...,2.772589,2.197225,0.000000,1.791759,0.693147,125388,2010-04-10,This was a very good Asian dressing. I sered ...,5,from www.topsecretrecipes.com i found this on ...
2,30037,seafood,bbq salmon filet,10,40,387.9,22.0,1.0,7.0,108.0,...,0.000000,1.945910,4.682131,2.397895,0.000000,222478,2010-04-10,Yum! Only change I made was to use a little le...,5,"lovely tender, juicy salmon bbq'd to perfectio..."
3,276596,vegetables,bavarian red cabbage,8,495,208.1,13.0,89.0,28.0,5.0,...,4.488636,3.332205,1.609438,3.295837,2.302585,228458,2010-04-10,Oh wow...this was so so so delicious! It does ...,5,a vegetarian delight with the ease of the croc...
4,224279,meat,grilled ham and swiss sandwich with grilled on...,11,20,398.9,35.0,28.0,34.0,25.0,...,3.332205,3.526361,3.218876,4.158883,2.484907,162826,2010-04-10,I want a 10 stars button for this one!\nOH MY ...,5,this is a favorite i made up while working in ...


In [26]:
X_test = test_data[['category', 'log calories', 'log fat',
       'log sugar', 'log sodium', 'log protein', 'log sat fat', 'log carbs']]
X_test = pd.concat([X_test, pd.get_dummies(X_test.category, prefix='Cat_')], axis=1).drop('category', axis = 1)

In [27]:
test_data['groups'] = rbf.predict(X_test)

In [28]:
test_data.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description,groups
0,336446,desserts,towboat brownies,8,40,363.4,25.0,135.0,14.0,10.0,...,2.639057,2.302585,3.713572,2.772589,526666,2010-04-10,"Cakey goodness, and oh so simple. I creamed th...",5,from the new orleans times picayune,3
1,47441,salads,tsr version of islands china coast salad dress...,6,7,101.2,12.0,16.0,9.0,0.0,...,2.197225,0.000000,1.791759,0.693147,125388,2010-04-10,This was a very good Asian dressing. I sered ...,5,from www.topsecretrecipes.com i found this on ...,1
2,30037,seafood,bbq salmon filet,10,40,387.9,22.0,1.0,7.0,108.0,...,1.945910,4.682131,2.397895,0.000000,222478,2010-04-10,Yum! Only change I made was to use a little le...,5,"lovely tender, juicy salmon bbq'd to perfectio...",1
3,276596,vegetables,bavarian red cabbage,8,495,208.1,13.0,89.0,28.0,5.0,...,3.332205,1.609438,3.295837,2.302585,228458,2010-04-10,Oh wow...this was so so so delicious! It does ...,5,a vegetarian delight with the ease of the croc...,1
4,224279,meat,grilled ham and swiss sandwich with grilled on...,11,20,398.9,35.0,28.0,34.0,25.0,...,3.526361,3.218876,4.158883,2.484907,162826,2010-04-10,I want a 10 stars button for this one!\nOH MY ...,5,this is a favorite i made up while working in ...,2


In [29]:
test_data.to_csv("test_healthy.csv")

### Evaluate test dataset to check whether the health rank is correct

In [40]:
import random
n = random.randint(0,len(test_data)-1)
recipe = test_data.iloc[n]
print(recipe)

recipe_id                                                         138352
category                                                         seafood
name                                              oven fried sesame fish
n_ingredients                                                          6
minutes                                                               45
calories                                                           223.8
total fat (PDV)                                                      7.0
sugar (PDV)                                                          0.0
sodium (PDV)                                                         5.0
protein (PDV)                                                       79.0
saturated fat (PDV)                                                  4.0
carbohydrates (PDV)                                                  0.0
log calories                                                    5.410753
log fat                                            